# Examples for EPTools
@copyright by Runduo Liang (Aujust)

## Crossmatch

In [15]:
import EPTools
from astropy.cosmology import Planck18
from astropy.coordinates import SkyCoord
import EPTools.utils

In [17]:
f = 2e-12

dL = Planck18.luminosity_distance(0.09).value * 1e6

print('lum: {} erg/s'.format(EPTools.utils.flx2lum(f,dL)))

lum: 4.330333114875513e+43 erg/s


In [12]:
crossmatch = EPTools.Crossmatch()

pos = '11h44m03.1835850144s +05d47m44.014153920s'

data = crossmatch.xmm_archive(pos)
print(data)

Welcome to ASAS-SN Skypatrol!

Current Deployment Version: 0.6.17 (26 JAN 2024)
Please upgrade your client if not up to date.

<No columns>
